# Step 1: Import and Init

In [ ]:
import os
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import sys
import imageio

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

# Step 2: Retrieve Data

In [ ]:
# CSV file path
CSV_PATH = "Crash_Table.csv"

# Load the CSV file into a DataFrame
df = pd.read_csv(CSV_PATH)

arr = []

for ind in df.index:
    for i in range(1, 51):
        arr.append(df[f"frame_{i}"][ind])

print(len(arr))
df

In [ ]:
IMAGE_PATH = "C:\\\\Users\\sacha\\OneDrive\\Bureau\\CrashBest"

# Load images from the directory and create a dataset
dataset = keras.preprocessing.image_dataset_from_directory(IMAGE_PATH, labels=arr, batch_size=128, label_mode='int')

# Step 3: Preparing the Data

In [ ]:
# Normalize the images to the range [0, 1]
dataset = dataset.map(lambda x, y: (x / 255.0, y))

In [ ]:

# Split the dataset into training (80%) and testing (20%)
train_size = int(len(dataset) * 0.8)

# Shuffle the dataset before splitting
dataset = dataset.shuffle(buffer_size=len(dataset))

# Split the dataset into training and testing sets
train_ds = dataset.take(train_size)
test_ds = dataset.skip(train_size)

# Step 4: Build a Model

In [ ]:
FILTER = 8

# Define the model architecture
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=(256, 256, 3)))

#Block 1
model.add(keras.layers.Conv2D(FILTER, (3, 3), padding="same", activation="relu"))
model.add(keras.layers.Conv2D(FILTER, (3, 3), padding="same", activation="relu"))
model.add(keras.layers.Reshape((256, 256, FILTER)))
model.add(keras.layers.MaxPooling2D((4, 4)))

#Block 2
model.add(keras.layers.Conv2D(FILTER * 2, (3, 3), padding="same", activation="relu"))
model.add(keras.layers.Conv2D(FILTER * 2, (3, 3), padding="same", activation="relu"))
model.add(keras.layers.MaxPooling2D((4, 4)))

#Block 3
model.add(keras.layers.Conv2D(FILTER * 4, (3, 3), padding="same", activation="relu"))
model.add(keras.layers.Conv2D(FILTER * 4, (3, 3), padding="same", activation="relu"))
model.add(keras.layers.MaxPooling2D((2, 2)))

# Flatten the output of the last convolutional layer
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(128, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss="binary_crossentropy", metrics = ['binary_accuracy'])
model.summary()

# Step 5: Train the model

In [ ]:
history = model.fit(train_ds, epochs=6, verbose=1, validation_data=test_ds)

In [ ]:
fig, ax1 = plt.subplots()

# Plot loss on the primary y-axis
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='tab:blue')
ax1.plot(history.history['loss'], label='Loss', color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

# Create a secondary y-axis for accuracy
ax2 = ax1.twinx()
ax2.set_ylabel('Accuracy', color='tab:orange')
ax2.plot(history.history['accuracy'], label='Accuracy', color='tab:orange')
ax2.tick_params(axis='y', labelcolor='tab:orange')

# Add a title and show the plot
plt.title('Training Loss and Accuracy')
fig.tight_layout()
plt.show()

# Step 6 : Evaluation 

In [ ]:
score = model.evaluate(test_ds, verbose=0)

print(f'Test loss     : {score[0]:4.4f}')
print(f'Test accuracy : {score[1]:4.4f}')

In [ ]:
y_sigmoid = model.predict(train_ds)
y_pred    = np.argmax(y_sigmoid, axis=-1)

from sklearn.metrics import classification_report,confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
# import scikitplot as skplt 

print(classification_report(y_test,y_pred,digits = 4))
print(accuracy_score(y_test,y_pred))

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=range(3), yticklabels=range(3))
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Step 7 : Save the model

In [ ]:
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model.save(f"CarCrashTensorflow_{timestamp}.keras")

# Step 8 : Load and try the model with new data

In [ ]:
model_path = "CarCrashTensorflow_20231003_123456.keras"  # Replace with your actual path

model2 = keras.models.load_model(model_path)

In [ ]:
anim_file = 'test.gif'
imgs = []
imgs[0].save(anim_file, save_all=True, append_images=imgs[1:], duration=100, loop=0)

In [2]:
NewData = cv2.cvtColor(cv2.imread('NewCrash.jpg'), cv2.COLOR_BGR2RGB)
plt.imshow(NewData)

NameError: name 'cv2' is not defined

In [ ]:
NewData = cv2.resize(NewData,(256,256))
plt.imshow(NewData)

In [ ]:
NewData.shape

In [ ]:
y_sigmoid = model2.predict(NewData)
y_pred    = np.argmax(y_sigmoid, axis=-1)

if y_pred == 0:
    print(f"No crash detected with {y_sigmoid}%") 
else:
    print(f"Crash detected with {y_sigmoid}%")